In [19]:
%pip install scipy
%pip install h5py
%pip install pandas




Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement requirements (from versions: none)
ERROR: No matching distribution found for requirements


  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [57]:
import scipy
import h5py
import pandas as pd
import roc

In [69]:
#Running Delong test on the data for the CT Variability Data

# 3 datasets to load in at three location
mat_file_FD=r"C:\Users\chirr\Downloads\AUC_results\1.Experiment_1\5.FP5_ALL_pruning\CT_TI_3Dresampled_grp2_FD_validation_10_wilcoxon_ALL_Randomforest.mat"
mat_file_HD=r"C:\Users\chirr\Downloads\AUC_results\2.Experiment_2\5.FP5_ALL_pruning_HD\CT_TI_3Dresampled_grp2_HD_validation_10_mrmr_ALL_Randomforest.mat"
mat_file_S4=r"C:\Users\chirr\Downloads\AUC_results\2.Experiment_2\5.FP5_ALL_pruning_S4\CT_TI_3Dresampled_grp2_S4_validation_10_mrmr_ALL_Randomforest.mat"
df=pd.read_csv(r'..\CT_grp2_labels.csv')
print(df.head)



<bound method NDFrame.head of     Labels  PatNums
0        0        1
1        0        2
2        0        3
3        0        4
4        1        5
..     ...      ...
69       1       70
70       1       71
71       1       72
72       1       73
73       1       74

[74 rows x 2 columns]>


In [42]:
import numpy as np
def print_struct(struct, indent=0):
    # Iterate over the fields of the struct
    for key in struct.dtype.names:
        value = struct[key]
        print(' ' * indent + f'{key}:', end=' ')
        if isinstance(value, np.ndarray):
            if value.ndim == 0:
                print(value.item())
            elif value.ndim == 1:
                print(value)
            elif value.ndim == 2:
                print(value)
            else:
                print("Array with shape", value.shape)
        elif isinstance(value, dict):
            print()
            print_struct(value, indent + 4)
        else:
            print(value)

In [62]:
def mat_to_stats(mat_data,training,labels):
    '''
      Returns:
      A dictionary of comparisons each containing a
        A named tuple with fields:
      effect: The estimated difference in the AUC-ROC between the
        two scores. A positive value means that y_score_2 is more discriminating
        than y_score_1.
      ci: A (lower, upper) confidence interval for the true difference in AUCs.
      statistic: The value of the z-statistic.
      pvalue: The p-value associated with the test. Unless a non-inferiority
        is specified, this is two-tailed.
        '''

    predictions_X_FD = mat_data['stats1']['prediction']
    predictions_X_HD = mat_data['stats2']['prediction']
    predictions_X_S4 = mat_data['stats3']['prediction']
    results={}
    results['Training']=str(training)
    results['FDvsHD']=roc.delong_test(y_true=labels.to_numpy(),y_score_1=predictions_X_FD[0][0],y_score_2=predictions_X_HD[0][0])
    results['FDvsS4']=roc.delong_test(y_true=labels.to_numpy(),y_score_1=predictions_X_FD[0][0],y_score_2=predictions_X_S4[0][0])
    results['HDvsS4']=roc.delong_test(y_true=labels.to_numpy(),y_score_1=predictions_X_HD[0][0],y_score_2=predictions_X_S4[0][0])

    return results



In [70]:
FD_trained_data=scipy.io.loadmat(mat_file_FD)
HD_trained_data=scipy.io.loadmat(mat_file_HD)
S4_trained_data=scipy.io.loadmat(mat_file_S4)
labels=df['Labels']
#print(FD_trained_data.keys())
#print(FD_trained_data['stats1'])
#print_struct(FD_trained_data['stats1'])
#print("Fields in stats1:", FD_trained_data['stats1'].dtype.names)
#prediction_trainingset_testingset
predictions_FD_FD = FD_trained_data['stats1']['prediction']
predictions_FD_HD = FD_trained_data['stats2']['prediction']
predictions_FD_S4 = FD_trained_data['stats3']['prediction']

roc.delong_test(y_true=labels.to_numpy(),y_score_1=predictions_FD_FD[0][0],y_score_2=predictions_FD_S4[0][0])

#key stats1 results on Fulldose
#key stats2 results on halfdose
#key stats3 results on Safire4
#Regardless of the source dataset
#Each row is a run for some reason

TestResult(effect=np.float64(-0.09685672514619903), ci=(np.float64(-0.1665525586062052), np.float64(-0.027160891686192873)), statistic=np.float64(-2.723773911907937), pvalue=np.float64(0.00645406717463634))

In [71]:
results=mat_to_stats(FD_trained_data,'Full Dose',labels)
print(results['Training'])
print(results['FDvsHD'])
print(results['FDvsS4'])
print(results['HDvsS4'])



Full Dose
TestResult(effect=np.float64(-0.05336257309941528), ci=(np.float64(-0.12228287150906998), np.float64(0.015557725310239426)), statistic=np.float64(-1.5175314647591331), pvalue=np.float64(0.12913255558473427))
TestResult(effect=np.float64(-0.09685672514619903), ci=(np.float64(-0.1665525586062052), np.float64(-0.027160891686192873)), statistic=np.float64(-2.723773911907937), pvalue=np.float64(0.00645406717463634))
TestResult(effect=np.float64(-0.04349415204678375), ci=(np.float64(-0.10456827251120737), np.float64(0.01757996841763986)), statistic=np.float64(-1.3957953205312648), pvalue=np.float64(0.16277613993848405))


In [72]:
results=mat_to_stats(HD_trained_data,'Half Dose',labels)
print(results['Training'])
print(results['FDvsHD'])
print(results['FDvsS4'])
print(results['HDvsS4'])

Half Dose
TestResult(effect=np.float64(-0.062134502923976376), ci=(np.float64(-0.13511241747753372), np.float64(0.010843411629580982)), statistic=np.float64(-1.6687430529262235), pvalue=np.float64(0.09516831289582134))
TestResult(effect=np.float64(-0.06140350877192968), ci=(np.float64(-0.12678379370679138), np.float64(0.00397677616293203)), statistic=np.float64(-1.8407485656765594), pvalue=np.float64(0.06565841333892643))
TestResult(effect=np.float64(0.0007309941520466934), ci=(np.float64(-0.07320873610955321), np.float64(0.0746707244136466)), statistic=np.float64(0.01937689258335028), pvalue=np.float64(0.9845404439951647))


In [73]:
results=mat_to_stats(S4_trained_data,' Safire 4',labels)
print(results['Training'])
print(results['FDvsHD'])
print(results['FDvsS4'])
print(results['HDvsS4'])

 Safire 4
TestResult(effect=np.float64(-0.08589181286549707), ci=(np.float64(-0.14277368458112216), np.float64(-0.029009941149871973)), statistic=np.float64(-2.9595520454188047), pvalue=np.float64(0.003080866574389241))
TestResult(effect=np.float64(-0.03252923976608191), ci=(np.float64(-0.09022757440099814), np.float64(0.025169094868834327)), statistic=np.float64(-1.1049909635937143), pvalue=np.float64(0.26916350470579264))
TestResult(effect=np.float64(0.05336257309941517), ci=(np.float64(-0.0060541461765114685), np.float64(0.1127792923753418)), statistic=np.float64(1.76025742706422), pvalue=np.float64(0.07836416900205345))
